In [2]:
import xnat
import pyxnat
from pyxnat import Interface
import xnat
import os
import pydicom
import tempfile
from tqdm import tqdm


In [ ]:
# #The URL of your XNAT
# xnat_host = os.environ['XNAT_HOST']

# #A temporary token which can be used as a username
# user = os.environ['XNAT_USER']

# #A temporary token which can be used as a password
# pwd = os.environ['XNAT_PASS']

In [ ]:
# session = xnat.connect(xnat_host, user=user, password=pwd)

In [60]:
session = xnat.connect('https://xnat.crmbm.univ-amu.fr', user='hdary')

In [61]:
my_project = session.projects["VIDA_BRAIN_EPINOV"]

check tag dicom for gender

check tag dicom for age

In [62]:
my_project.subjects

<XNATListing {(XNAT_CRMBM_S00019, 01_47_Q-M): <SubjectData 01_47_Q-M (XNAT_CRMBM_S00019)>, (XNAT_CRMBM_S00020, 01_39_DT): <SubjectData 01_39_DT (XNAT_CRMBM_S00020)>, (XNAT_CRMBM_S00021, 01-68-P-N): <SubjectData 01-68-P-N (XNAT_CRMBM_S00021)>, (XNAT_CRMBM_S00040, 01-38-F-G): <SubjectData 01-38-F-G (XNAT_CRMBM_S00040)>, (XNAT_CRMBM_S00004, 01-86-pl): <SubjectData 01-86-pl (XNAT_CRMBM_S00004)>, (XNAT_CRMBM_S00008, 01-85-g-s): <SubjectData 01-85-g-s (XNAT_CRMBM_S00008)>, (XNAT_CRMBM_S00009, 01-75-rs): <SubjectData 01-75-rs (XNAT_CRMBM_S00009)>, (XNAT_CRMBM_S00010, 01-72-m-v): <SubjectData 01-72-m-v (XNAT_CRMBM_S00010)>, (XNAT_CRMBM_S00011, 01-42-A-S): <SubjectData 01-42-A-S (XNAT_CRMBM_S00011)>, (XNAT_CRMBM_S00012, 01_73_A-M): <SubjectData 01_73_A-M (XNAT_CRMBM_S00012)>, (XNAT_CRMBM_S00013, 01-61-B-L): <SubjectData 01-61-B-L (XNAT_CRMBM_S00013)>, (XNAT_CRMBM_S00014, 01-64-B-A): <SubjectData 01-64-B-A (XNAT_CRMBM_S00014)>, (XNAT_CRMBM_S00015, 01_41_CL): <SubjectData 01_41_CL (XNAT_CRMBM_S00

In [63]:
subject = my_project.subjects['01-38-F-G']
experiment = subject.experiments['202011161412']
scan = experiment.scans[0]

In [ ]:
scan.dicom_dump()

In [7]:
file_obj = scan.resources['DICOM'].files[0]

In [8]:
with file_obj.open() as fin:
    data = pydicom.dcmread(fin, stop_before_pixels=True)
    patient_birth_date = data[0x010,0x030].value
    sex = data[0x010,0x040].value
    age = data[0x010,0x1010].value
    size = data[0x010,0x1020].value
    weight = data[0x010,0x1030].value
    unique_id = data[0x008,0x050].value


In [9]:
print(patient_birth_date)
print(sex)
print(age)
print(size)
print(weight)
print(unique_id)

19820913
M
038Y
1.82
86



In [64]:
subject.custom_variables['default']['patient_unique_id'] = unique_id

<CustomVariableGroup default {patient_unique_id (string): None}>

In [65]:
experiment.custom_variables['default']['patient_unique_id'] = unique_id

Exception in thread XNATpyKeepAliveThread:
Traceback (most recent call last):
  File "/home/hdary/miniconda3/lib/python3.8/site-packages/urllib3/connection.py", line 159, in _new_conn
    conn = connection.create_connection(
  File "/home/hdary/miniconda3/lib/python3.8/site-packages/urllib3/util/connection.py", line 61, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/home/hdary/miniconda3/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hdary/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 670, in urlopen
    httplib_response = self._make_request(
  File "/home/hdary/miniconda3/lib/python3.8/site-packages/urllib3/connectionpool.py", line 381, in _ma

In [94]:
subject.demographics.age = int(age.replace('Y',''))
# subject.demographics.height = size
# subject.demographics.weight = weight
subject.demographics.yob = patient_birth_date
subject.demographics.gender = sex.replace('M','male').replace('F','female')


script 

In [95]:
my_project = session.projects["VIDA_BRAIN_EPINOV"]

for s in tqdm(my_project.subjects.values()):
    print(s.id, s.label)

    # if s.demographics.yob != None:
    #     continue
    
    e = s.experiments[0]
    scan = e.scans[0]
    file_obj = scan.resources['DICOM'].files[0]

    with file_obj.open() as fin:
        data = pydicom.dcmread(fin, stop_before_pixels=True)
        patient_birth_date = data[0x010,0x030].value
        sex = data[0x010,0x040].value
        age = data[0x010,0x1010].value
        size = data[0x010,0x1020].value
        weight = data[0x010,0x1030].value

        try:
            s.demographics.age = int(age.replace('Y',''))
        except ValueError as e:
            print('Error setting age:', e)

        # try:
        #     # s.demographics.height = size
        #     pass
        # except Exception as e:
        #     print('Error setting height:', e)

        # try:
        #     # s.demographics.weight = weight
        #     pass
        # except Exception as e:
        #     print('Error setting weight:', e)

        try:
            s.demographics.yob = patient_birth_date
        except ValueError as e:
            print('Error setting year of birth:', e)

        try:
            s.demographics.gender = sex.replace('M','male').replace('F','female')
        except ValueError as e:
            print('Error setting gender:', e)

  0%|          | 0/52 [00:00<?, ?it/s]

XNAT_CRMBM_S00019 01_47_Q-M


  2%|▏         | 1/52 [00:05<04:15,  5.01s/it]

XNAT_CRMBM_S00020 01_39_DT


  4%|▍         | 2/52 [00:09<03:53,  4.67s/it]

...

 96%|█████████▌| 50/52 [03:36<00:07,  3.89s/it]

XNAT_CRMBM_S00041 01-65-S-F


 98%|█████████▊| 51/52 [03:41<00:04,  4.02s/it]

XNAT_CRMBM_S00042 01-66-L-Q


100%|██████████| 52/52 [03:45<00:00,  4.34s/it]
